# Coroutine(코루틴)

In [1]:
# 함수를 호출한 뒤 함수가 끝나면 현재 코드로 다시 돌아온다.
# 함수에 들어있던 변수와 계산식은 모두 사라진다

def add(a, b):
    c = a + b    # add 함수가 끝나면 변수와 계산식은 사라짐
    print(c)
    print('add 함수')

def calc():
    add(1, 2)    # add 함수가 끝나면 다시 calc 함수로 돌아옴
    print('calc 함수')

calc()

3
add 함수
calc 함수


#### 메인루틴과 서브루틴의 동작과정
메인루틴 (호출한측),  서브루틴 (호출받은쪽)<br>
![](https://dojang.io/pluginfile.php/13976/mod_page/content/3/041001.png)

메인 루틴에서 서브 루틴을 호출하면 서브 루틴의 코드를 실행한 뒤 다시 메인 루틴으로 돌아옵니다. 특히 서브 루틴이 끝나면 서브 루틴의 내용은 모두 사라집니다. 즉, 서브 루틴은 메인 루틴에 '종속된 관계'입니다.

#### 코루틴은 다르다
코루틴(coroutine)은 cooperative routine를 의미하는데 서로 협력하는 루틴이라는 뜻입니다. 즉, 메인 루틴과 서브 루틴처럼 종속된 관계가 아니라 서로 대등한 관계이며 특정 시점에 상대방의 코드를 실행합니다.

![](https://dojang.io/pluginfile.php/13976/mod_page/content/3/041002.png)

코루틴은 함수가 종료되지 않은 상태에서 메인 루틴의 코드를 실행한 뒤 다시 돌아와서 코루틴의 코드를 실행합니다. 따라서 코루틴이 종료되지 않았으므로 코루틴의 내용도 계속 유지됩니다

일반 함수를 호출하면 코드를 한 번만 실행할 수 있지만, 코루틴은 코드를 여러 번 실행할 수 있습니다. 참고로 함수의 코드를 실행하는 지점을 진입점(entry point)이라고 하는데, 코루틴은 진입점이 여러 개인 함수

## 코루틴에 값 보내기 + 받기
- 코루틴은 제너레이터의 특별한 형태
- 코루틴은 yield로 값을 받아올 수 있습니다  (*제너레이터는 yield로 값을 발생시켰었다)
- 코루틴에 값을 보내면서 코드를 실행할 때 : **send() 메소드** 사용
    - 코루틴객체.send(값)
- send() 메소드가 보낸 값을 받아오려면 **(yield)**  <- 괄호 꼭!    
    - 변수 = (yield)

In [5]:
def number_routine():
    print("코루틴 최초 진입")
    while True:
        x = (yield)
        print(x)

In [6]:
co = number_routine()
next(co)  #코루틴 안의 yield 까지 코드 실행 (최초 실행)

코루틴 최초 진입


In [7]:
co.send(1)  #코루틴에 숫자 1을 보냄

1


In [8]:
co.send(2)

2


In [9]:
co.send(3)

3


#### 위 코루틴의 동작 과정

![](https://dojang.io/pluginfile.php/13976/mod_page/content/3/041003.png)

### 코루틴 코드를 최초로 실행하는 다양한 방법들

In [10]:
co = number_routine()
next(co)

co.send(50)

코루틴 최초 진입
50


In [11]:
co = number_routine()
co.__next__()

co.send(10)
co.send(20)

코루틴 최초 진입
10
20


In [12]:
co = number_routine()
co.send(None)

co.send(10)
co.send(20)
co.send(30)

코루틴 최초 진입
10
20
30


---
## 코루틴 바깥으로 값 전달하기

- (yield 변수)

 yield에 변수를 지정한 뒤 괄호로 묶어주면 값을 받아오면서 바깥으로 값을 전달합니다. 그리고 yield를 사용하여 바깥으로 전달한 값은 'next() 함수(__next__ 메서드)'와 'send() 메소드'의 **리턴값**으로 나옵니다.

In [16]:
def sum_coroutine():
    total = 0
    while True:
        x = (yield total)  # 코루틴 바깥에서 값을 받아오면서, 바깥으로 값을 전달
        total += x
        
co = sum_coroutine()
print(next(co))  # 0: 코루틴 안의 yield 까지 코드를 실행하고 코루틴에서 나온 값 출력

print(co.send(1))
print(co.send(5))
print(co.send(3))


0
1
6
9


#### 값을 전달하는 과정
![](https://dojang.io/pluginfile.php/13977/mod_page/content/5/041005.png)

### generator 와 cotoutine 의 차이점 정리
- 제너레이터는 next 함수(__next__ 메서드)를 반복 호출하여 값을 얻어내는 방식
- 코루틴은 next 함수(__next__ 메서드)를 한 번만 호출한 뒤 send로 값을 주고 받는 방식
- 코루틴도 결국은 제네레이터 다


※ 참고: 값을 보내지 않고 코루틴의 코드 실행하기<br>
값을 보내지 않으면서 코루틴의 코드를 실행할 때는 next 함수(__next__ 메서드)만 사용하면 됩니다. 잘 생각해보면 이 방식이 일반적인 제너레이터입니다.

---
##  코루틴을 종료하고 예외 처리하기
**코루틴객체.close()**

보통 코루틴은 실행 상태를 유지하기 위해 while True:를 사용해서 끝나지 않는 무한 루프로 동작합니다. 만약 코루틴을 강제로 종료하고 싶다면 close 메서드를 사용합니다.



In [19]:
def number_coroutine():
    while True:
        x = (yield)
        print(x, end = ' ')
        
        
co = number_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
co.close()  # 코루틴 종료

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

In [20]:
co.send(10)

StopIteration: 

### GeneratorExit 예외 처리
close() 메소드를 호출하면 코루틴이 종료될 때 **GeneratorExit** 예외가 발생합니다.<br>
따라서 이 예외를 처리하면 코루틴의 종료 시점에서 수행할 코루틴의 작업들 수행 가능

In [23]:
def number_coroutine():
    try:
        while True:
            x = (yield)
            print(x, end = ' ')
    except GeneratorExit:  # 코루틴이 종료될때 GeneratorExit 예외 발생
        print()
        print('코루틴 종료')
        
        
co = number_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
co.close()  # 코루틴 종료
print('프로그램 종료')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
코루틴 종료
프로그램 종료


### 코루틴 안에서 예외 발생시키기
**코루틴객체.throw(예외이름, 에러메시지)**


In [24]:
# 코루틴 안에 예외를 발생 시킬 때는 throw 메서드를 사용합니다. 
# throw는 말그대로 던지다라는 뜻인데 예외를 코루틴 안으로 던집니다. 
# 이때 throw 메서드에 지정한 에러 메시지는 except as의 변수에 들어갑니다.

In [27]:
def sum_coroutine():
    print('코루틴 시작')
    try:
        total = 0
        while True:
            x = (yield)
            total += x
    except RuntimeError as e:
        print(e)
        yield total  # 코루틴 바깥ㄴ으로 전달됨 throw() 의 리턴값이 된다.
    print('코루틴 종료')
        
co = sum_coroutine()
next(co)

for i in range(20):
    co.send(i)
    
print(co.throw(RuntimeError, '예외로 코루틴 끝내기'))

print('프로그램 종료')

코루틴 시작
예외로 코루틴 끝내기
190
프로그램 종료


## 하위 코루틴의 리턴값 가져오기
**변수 = yield from 코루틴()**  (파이썬 3.3 <= 사용)

In [33]:
def accumulate():
    print("accumulate() 시작")
    total = 0
    while True:
        x = (yield)
        if x is None:
            return total
        total += x
        
def sum_coroutine():
    print("sum_couroutin() 시작")
    while True:
        total = yield from accumulate()   # accumulate 의 반환값을 가져옴
        print(total)
        
co = sum_coroutine()
next(co)

for i in range(1, 11):  
    co.send(i)
    
co.send(None)

for i in range(1, 101):
    co.send(i)
    
co.send(None)


sum_couroutin() 시작
accumulate() 시작
55
accumulate() 시작
5050
accumulate() 시작


#### StopIteration 예외 발생

In [35]:
def accumulate():
    print("accumulate() 시작")
    total = 0
    while True:
        x = (yield)
        if x is None:
            raise StopIteration(total)
            return total
        total += x
        
def sum_coroutine():
    print("sum_couroutin() 시작")
    while True:
        total = yield from accumulate()   # accumulate 의 반환값을 가져옴
        print(total)
        
co = sum_coroutine()
next(co)

for i in range(1, 11):  
    co.send(i)
    
co.send(None)

for i in range(1, 101):
    co.send(i)
    
co.send(None)


sum_couroutin() 시작
accumulate() 시작


RuntimeError: generator raised StopIteration

## 연습 : 문자열 검색 코루틴 만들기

In [36]:
# 다음 소스 코드를 완성하여 문자열에서 특정 단어가 있으면 True, 
# 없으면 False가 출력되게 만드세요. find 함수는 코루틴으로 작성해야 합니다.

def find(word):
    result = False
    while True:
        line = (yield result)
        result = word in line    
                                 
f = find('Python')
next(f)
 
print(f.send('Hello, Python!'))
print(f.send('Hello, world!'))
print(f.send('Python Script'))
 
f.close()

# [실행결과]
# True
# False
# True

True
False
True
